In [1]:
from tqdm import tqdm

In [2]:
def read_csv(path):
    result = []

    with open(path) as f:
        for line in f.readlines():
            result.append(line)

    return result


In [3]:
predictions_path = "../../tmp/tst-csc-test/generated_predictions.txt"
target_path = "../../data/rawdata/ctc2021/valid.through"

predictions = read_csv(predictions_path)    
target = read_csv(target_path)


In [4]:
for i in range(10):
    print(predictions[i], "%"*2, target[i])

6 7 玮
 %% 18 i 覆了 26 i 模 84 i 用。

9 10 友
 %% 27 28 不 29 30 忍


 %% 5 i e 33 35 食用 55 57 药效

2 3 起 16 17 你
 %% 1 2 便 12 13 时 26 28 德军 48 49 并 56 57 居

13 14 舞
 %% 6 7 后


 %% 75 i e 76 i e


 %% 

3 4 餐
 %% 4 5 府

0 1 走
 %% 9 10 d 13 14 d


 %% 8 i 方 14 i 方 16 17 其 18 i 偶



In [5]:
def build_structure(throughs):
    """
    #turn ['1 2 啥'] to {('1', '2'):'啥'}
    #
    #
    """

    result = []

    for through in throughs:

        through = through.split()

        pointer = 0

        catch = {}

        #c++ style code for adapt "2 3 好 的" 
        while pointer + 3 <= len(through):

            key = (through[pointer], through[pointer+1])# key like : (1,2) 
        
            catch[key] = through[pointer+2]

            while pointer + 3 < len(through) and not through[pointer+3].isdigit():

                pointer += 1

                catch[key] += through[pointer+2]
            
            pointer += 3

        result.append(catch)

    return result


In [6]:
structured_predictions, structured_target = build_structure(predictions), build_structure(target)

In [7]:
for i in range(10):
    print(structured_predictions[i])
    print(structured_target[i])

{('6', '7'): '玮'}
{('18', 'i'): '覆了', ('26', 'i'): '模', ('84', 'i'): '用。'}
{('9', '10'): '友'}
{('27', '28'): '不', ('29', '30'): '忍'}
{}
{('5', 'i'): 'e', ('33', '35'): '食用', ('55', '57'): '药效'}
{('2', '3'): '起', ('16', '17'): '你'}
{('1', '2'): '便', ('12', '13'): '时', ('26', '28'): '德军', ('48', '49'): '并', ('56', '57'): '居'}
{('13', '14'): '舞'}
{('6', '7'): '后'}
{}
{('75', 'i'): 'e', ('76', 'i'): 'e'}
{}
{}
{('3', '4'): '餐'}
{('4', '5'): '府'}
{('0', '1'): '走'}
{('9', '10'): 'd', ('13', '14'): 'd'}
{}
{('8', 'i'): '方', ('14', 'i'): '方', ('16', '17'): '其', ('18', 'i'): '偶'}


In [23]:
### f1 score
###
#《Introduction to SIGHAN 2015 Bake-off for Chinese Spelling Check》
# https://aclanthology.org/W15-3106.pdf
#
# only for detection
##

def get_score(predict, target, debug = False):

    tp, fp, fn, tn = 0, 0, 0, 0

    tp_cor, fp_cor, fn_cor, tn_cor = 0, 0, 0, 0

    from operator import eq

    for i in tqdm(range(len(predict))):

        dict_pre, dict_true = predict[i], target[i]

        for key in dict_pre.keys():
        
            weight_pre = 1

            if key in dict_true.keys():
                """
                Picked and Need Pick
                """
                tp += weight_pre
                if eq(dict_true[key], dict_pre[key]):
                    tp_cor += weight_pre
                else:
                    fn_cor += weight_pre
                    fn += weight_pre
            else:
                """
                Picked but Dont Need Pick
                """
                fp += weight_pre

        """
        Unpick but Need Pick
        """
        for key in dict_true.keys():
            
            weight_true = 1
            
            if key not in dict_pre.keys():
                fn += weight_true

        """
        Unpick and Dont Need Pick
        """

        sub_seq = []

        ###
        #since f1 score dont need fn, so we wont calculate it
        ###
        
    print("TP: ",tp, "FP: ", fp, "FN: ", fn,)
    
    precision = tp / (tp + fp + 1e-10)

    recall = tp / (tp + fn + 1e-10)

    F1_score = 2 * precision * recall / (precision + recall + 1e-10)

    print("Precision: ", precision, "Recall: ", recall)

    print("F1_score: ", F1_score)

    return F1_score

get_score(structured_predictions, structured_target, False)




100%|██████████| 5000/5000 [00:00<00:00, 171624.79it/s]

TP:  4536 FP:  3456 FN:  7174
Precision:  0.5675675675675604 Recall:  0.38736122971818626
F1_score:  0.46046086686884014


0.46046086686884014

In [10]:
### f1 score
###
#《Introduction to SIGHAN 2015 Bake-off for Chinese Spelling Check》
# https://aclanthology.org/W15-3106.pdf
#
##

def get_score(predict, target, debug = False):
    """
    Detection 
    """
    print("*"*5, "Detection", "*"*5)

    tp, fp, fn, tn = 0, 0, 0, 0

    tp_list, fp_list, fn_list = [], [], []

    for i in tqdm(range(len(predict))):
        
        dict_pre, dict_true = predict[i], target[i]
        #print(dict_pre, dict_true)

        if not dict_true and dict_pre:
            """
            Picked but Dont Need Pick
            """
            fp += 1
            fp_list.append(i)
            continue
        
        if not dict_true and not dict_pre:
            continue

        flag = False
        for key in dict_true.keys():
            if not key in dict_pre.keys():
                fn += 1
                flag = True
                fn_list.append(i)
                break
        if flag:
            continue
         
        for key in dict_pre.keys():            
            if key not in dict_true.keys():
                """
                Need Pick but not Picked
                """
                fn += 1
                fn_list.append(i)
                break
        else:
            """
            Picked and Need Pick
            """
            tp += 1
            tp_list.append(i)

        """
        Unpick and Dont Need Pick
        """
        #since f1 score dont need fn, so we wont calculate it
        
    print("TP: ",tp, "FP: ", fp, "FN: ", fn,)
    
    precision = tp / (tp + fp + 1e-10)

    recall = tp / (tp + fn + 1e-10)

    F1_score = 2 * precision * recall / (precision + recall + 1e-10)

    print("Precision: ", precision, "Recall: ", recall)

    print("F1_score: ", F1_score)

    return F1_score, tp_list, fp_list, fn_list

In [8]:
### f1 score
###
#《Introduction to SIGHAN 2015 Bake-off for Chinese Spelling Check》
# https://aclanthology.org/W15-3106.pdf
#
##

def get_score_correction(predict, target):
    """
    Correction Level
    """
    print("*"*5, "Correction", "*"*5)

    tp, fp, fn, tn = 0, 0, 0, 0

    tp_list, fp_list, fn_list = [], [], []

    for i in tqdm(range(len(predict))):

        dict_pre, dict_true = predict[i], target[i]
        #print(dict_pre, dict_true)

        if not dict_true and dict_pre:
            """
            Picked but Dont Need Pick
            """
            fp += 1
            fp_list.append(i)
            continue
        
        if not dict_true and not dict_pre:
            continue

        flag = False
        for key in dict_true.keys():
            if not key in dict_pre.keys():
                fn += 1
                flag = True
                fn_list.append(i)
                break
        if flag:
            continue

        for key in dict_pre.keys():            
            if key not in dict_true.keys():
                """
                Need Pick but Not Picked
                """
                fn += 1
                fn_list.append(i)
                break
            elif dict_pre[key] != dict_true[key]:
                fn += 1
                fn_list.append(i)
                break
        else:
            """
            Picked and Need Pick
            """
            tp += 1
            tp_list.append(i)
            
        #since f1 score dont need fn, so we wont calculate it
        
    print("TP: ",tp, "FP: ", fp, "FN: ", fn,)
    
    precision = tp / (tp + fp + 1e-10)

    recall = tp / (tp + fn + 1e-10)

    F1_score = 2 * precision * recall / (precision + recall + 1e-10)

    print("Precision: ", precision, "Recall: ", recall)

    print("F1_score: ", F1_score)

    return F1_score, tp_list, fp_list, fn_list

In [11]:
#ctc2021 test

predictions_path = "../../tmp/tst-csc-test/generated_predictions.txt"
target_path = "../../data/rawdata/ctc2021/valid.through"

predictions, target = read_csv(predictions_path), read_csv(target_path)

structured_predictions, structured_target = build_structure(predictions), build_structure(target)

_ = get_score(structured_predictions, structured_target)

_ = get_score_correction(structured_predictions, structured_target)

***** Detection *****


 45%|████▌     | 500/1100 [00:00<00:00, 277034.61it/s]


IndexError: list index out of range